In [23]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /home/nilsw/Lab1ID2223
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [24]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [25]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-14 13:59:33,368 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-14 13:59:33,384 INFO: Initializing external client
2025-11-14 13:59:33,384 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-14 13:59:34,875 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286328


'{"country": "Japan", "city": "saitama-ken", "street": "gakuendai", "aqicn_url": "https://api.waqi.info/feed/@9937", "latitude": "36.0226816", "longitude": "139.7228851"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [26]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality_v_cgrade',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_v_cgrade',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [27]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,63.0,Japan,saitama-ken,gakuendai,2025-11-14,https://api.waqi.info/feed/@9937


In [28]:
# NEW. NEED TO ADD ROLLING FROM PREVIOUS TO THIS DAY
# FETCH LAST THREE PM25s FROM df
import time
#query = air_quality_fg.select_all().filter(air_quality_fg["date"] >= datetime.date.today()-datetime.timedelta(days=4))

#df_recent = query.read()
#print(df_recent)
#aq_today_df["roll"] = df_recent["pm25"].mean().astype("float64")
#aq_today_df = aq_today_df[["date","pm25","roll","country","city","street","url"]]
#aq_today_df
query = air_quality_fg.select_all().filter(air_quality_fg['date']>= today-datetime.timedelta(days=4))

df_recent = query.read()

df_recent = df_recent.sort_values('date')
print(df_recent)
pm_prev3 = df_recent['pm25'].head(3)
aq_today_df['rolling3'] = pm_prev3.mean()

aq_today_df['lag1d'] = df_recent['pm25'].iloc[2]
aq_today_df['lag2d'] = df_recent['pm25'].iloc[1]
aq_today_df['lag3d'] = df_recent['pm25'].iloc[0]


aq_today_df 

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.60s) 
                       date  pm25   rolling3  lag1d  lag2d  lag3d country  \
1 2025-11-11 00:00:00+00:00  14.0  40.000000   30.0   48.0   42.0   Japan   
0 2025-11-12 00:00:00+00:00  19.0  30.666666   14.0   30.0   48.0   Japan   
2 2025-11-13 00:00:00+00:00  57.0  21.000000   19.0   14.0   30.0   Japan   

          city     street                               url  
1  saitama-ken  gakuendai  https://api.waqi.info/feed/@9937  
0  saitama-ken  gakuendai  https://api.waqi.info/feed/@9937  
2  saitama-ken  gakuendai  https://api.waqi.info/feed/@9937  


,pm25,country,city,street,date,url,rolling3,lag1d,lag2d,lag3d
0,63.0,Japan,saitama-ken,gakuendai,2025-11-14,https://api.waqi.info/feed/@9937,30.0,57.0,19.0,14.0


In [29]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   pm25      1 non-null      float32       
 1   country   1 non-null      object        
 2   city      1 non-null      object        
 3   street    1 non-null      object        
 4   date      1 non-null      datetime64[ns]
 5   url       1 non-null      object        
 6   rolling3  1 non-null      float32       
 7   lag1d     1 non-null      float32       
 8   lag2d     1 non-null      float32       
 9   lag3d     1 non-null      float32       
dtypes: datetime64[ns](1), float32(5), object(4)
memory usage: 192.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [30]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 36.0°N 139.75°E
Elevation 8.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-14,12.50,0.1,0.360000,270.000000,saitama-ken
1,2025-11-15,9.00,0.0,4.198285,120.963692,saitama-ken
2,2025-11-16,11.25,0.0,1.835647,101.309898,saitama-ken
3,2025-11-17,11.65,0.0,6.763786,334.798920,saitama-ken
4,2025-11-18,6.30,0.0,8.891344,338.629303,saitama-ken
5,2025-11-19,6.90,0.0,6.877790,83.991089,saitama-ken
6,2025-11-20,7.40,0.0,2.902413,29.744795,saitama-ken


In [31]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 356.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [32]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-14 13:59:42,059 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286328/fs/1273950/fg/1703435


Uploading Dataframe: 100.00% |██████████████████████████████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_v_cgrade_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286328/jobs/named/air_quality_v_cgrade_1_offline_fg_materialization/executions


(Job('air_quality_v_cgrade_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747672
         }
       },
       "result": {
         "observed_value": 63.000003814697266,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-14T12:59:42.000057Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
  

In [33]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-14 13:59:56,427 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286328/fs/1273950/fg/1703436


Uploading Dataframe: 100.00% |██████████████████████████████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_v_cgrade_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286328/jobs/named/weather_v_cgrade_1_offline_fg_materialization/executions
2025-11-14 14:00:13,812 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-11-14 14:00:16,982 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-14 14:01:39,986 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-14 14:01:40,175 INFO: Waiting for log aggregation to finish.
2025-11-14 14:01:48,760 INFO: Execution finished successfully.


(Job('weather_v_cgrade_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747673
         }
       },
       "result": {
         "observed_value": 0.35999998450279236,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-14T12:59:56.000426Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectat

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
